# Installation Notes for K8s (kind) and Kubeflow PIPELINES Locally on WSL Ubuntu

# Kubeflow Pipelines

## Docker
```
# Install Docker Desktop (for Windows or macOS)
# Enable WSL2 support in Docker settings (for Windows)
# https://docs.docker.com/desktop/windows/install/
# Docker installed inside WSL has internet connectivity issues with minikube (Use Docker Desktop instead)

$ sudo apt-get update
$ sudo apt-get upgrade

$ docker --version
```

## Kubernetes cluster (minikube)
https://kubernetes.io/docs/tutorials/hello-minikube/
```
# See another notebook for how to install minikube
minikube start --cpus 12 --memory 32g --disk-size 60g --driver=docker

watch -c minikube kubectl -- get po -A
minikube logs

#alias kubectl='minikube kubectl'

minikube dashboard

kubectl cluster-info
```

## Get Full Kubeflow (not working, see below)
https://github.com/kubeflow/manifests
```
# Kubeflow 1.4.0 is not compatible with the latest versions of of kustomize 4.x.
https://github.com/kubeflow/manifests#frequently-asked-questions

cd /home/ksatola/git/MLOps/notebooks/kubeflow_kustomization
git clone https://github.com/kubeflow/manifests
    
# Deployment takes several minutes
while ! kubectl kustomize build example | kubectl apply -k example/; do echo "Retrying to apply resources"; sleep 10; done

watch -c minikube kubectl -- get po -A

# Run the following to port-forward Istio's Ingress-Gateway to local port 8080:
kubectl port-forward svc/istio-ingressgateway -n istio-system 8080:80
    
#You can access the Kubeflow Central Dashboard by doing the following:
- Open your browser and visit http://localhost:8080. You should get the Dex login screen.
- Login with the default user's credential. The default email address is `user@example.com` and the default password is `12341234`.
```

## Kubeflow Pipelines (not working)
https://www.kubeflow.org/docs/components/pipelines/overview/pipelines-overview/
```
# env/platform-agnostic-pns hasn't been publically released, so you will install it from master
export PIPELINE_VERSION=1.7.0
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources?ref=$PIPELINE_VERSION"
minikube kubectl -- wait --for condition=established --timeout=60s crd/applications.app.k8s.io
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/env/platform-agnostic-pns?ref=$PIPELINE_VERSION"

minikube logs

watch -c minikube kubectl -- get po -A

minikube kubectl -- port-forward -n kubeflow svc/ml-pipeline-ui 8585:80
# Open: http://localhost:8585/

# https://kubernetes.io/docs/reference/kubectl/cheatsheet/
minikube kubectl -- get services
minikube kubectl -- get pods -n kubeflow


```

## Cleaning
```
minikube pause
minikube unpause
minikube stop

minikube delete --all

# Remove docker if installed inside WSL (Ubuntu)
docker system prune
# https://docs.docker.com/engine/install/ubuntu/
sudo apt-get purge docker-ce docker-ce-cli containerd.io
sudo rm -rf /var/lib/docker
sudo rm -rf /var/lib/containerd
```